# Model Comparison

This notebook compares different topic modeling algorithms available in NBTM:

| Model | Description | Key Feature |
|-------|-------------|-------------|
| **Gibbs LDA** | Collapsed Gibbs Sampling | Simple, interpretable |
| **LDA-VI** | Variational Inference | Fast, scalable |
| **HDP** | Hierarchical Dirichlet Process | Automatic topic count |
| **CTM** | Correlated Topic Model | Topic correlations |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from nbtm.models import create_model, get_available_models
from nbtm.evaluation import (
    compute_coherence,
    compute_topic_diversity,
    BenchmarkRunner,
)
from nbtm.visualization import (
    plot_model_comparison,
    plot_radar_comparison,
    plot_training_time_comparison,
)

## 1. Prepare Data

Generate synthetic documents with known topic structure.

In [ ]:
# Define topic vocabularies
topic_words = {
    "tech": ["machine", "learning", "algorithm", "data", "model", "neural", "network", "deep", "training", "optimization"],
    "science": ["research", "experiment", "hypothesis", "theory", "analysis", "evidence", "study", "method", "result", "conclusion"],
    "stats": ["probability", "distribution", "bayesian", "inference", "prior", "posterior", "likelihood", "estimation", "variance", "mean"],
}

np.random.seed(42)

def generate_document(topic_name, length=20):
    """Generate a document from a topic."""
    words = topic_words[topic_name]
    return list(np.random.choice(words, size=length, replace=True))

def generate_mixed_document(topics, weights, length=20):
    """Generate a document mixing multiple topics."""
    doc = []
    for _ in range(length):
        topic = np.random.choice(topics, p=weights)
        word = np.random.choice(topic_words[topic])
        doc.append(word)
    return doc

# Generate corpus
documents = []
topics = list(topic_words.keys())

# Pure topic documents
for topic in topics:
    for _ in range(10):
        documents.append(generate_document(topic))

# Mixed topic documents
for _ in range(20):
    weights = np.random.dirichlet([0.5] * len(topics))
    documents.append(generate_mixed_document(topics, weights))

print(f"Generated {len(documents)} documents")
print(f"Sample document: {documents[0][:10]}...")

## 2. Define Models

Create instances of each model type.

In [ ]:
# Model configurations
model_configs = {
    "Gibbs LDA": {
        "model_type": "lda_gibbs",
        "num_topics": 3,
        "alpha": 0.1,
        "beta": 0.01,
    },
    "Variational LDA": {
        "model_type": "lda_vi",
        "num_topics": 3,
        "alpha": 0.1,
        "beta": 0.01,
    },
    "HDP": {
        "model_type": "hdp",
        "num_topics": 10,  # Initial/max topics
        "alpha": 1.0,
        "beta": 0.01,
    },
    "CTM": {
        "model_type": "ctm",
        "num_topics": 3,
        "alpha": 0.1,
        "beta": 0.01,
    },
}

print("Models to compare:")
for name in model_configs:
    print(f"  - {name}")

## 3. Train and Evaluate

Train each model and collect metrics.

In [ ]:
import time

results = {}

for name, config in model_configs.items():
    print(f"\nTraining {name}...")
    
    # Create model
    model = create_model(
        config["model_type"],
        num_topics=config["num_topics"],
        alpha=config["alpha"],
        beta=config["beta"],
        random_state=42,
    )
    
    # Train with timing
    start_time = time.time()
    model.fit(documents, num_iterations=300)
    training_time = time.time() - start_time
    
    # Evaluate
    coherence = compute_coherence(model, documents, measure="umass")
    diversity = compute_topic_diversity(model, top_n=10)
    
    results[name] = {
        "model": model,
        "training_time": training_time,
        "coherence": coherence,
        "diversity": diversity,
        "num_topics": model.num_topics,
        "log_likelihood": model.log_likelihood(),
    }
    
    print(f"  Time: {training_time:.2f}s")
    print(f"  Topics: {model.num_topics}")
    print(f"  Coherence: {coherence:.4f}")
    print(f"  Diversity: {diversity:.4f}")

## 4. Compare Results

Visualize the comparison between models.

In [ ]:
# Create comparison table
print("\n" + "=" * 70)
print(f"{'Model':<20} {'Topics':<10} {'Time (s)':<12} {'Coherence':<12} {'Diversity':<12}")
print("=" * 70)

for name, res in results.items():
    print(f"{name:<20} {res['num_topics']:<10} {res['training_time']:<12.2f} {res['coherence']:<12.4f} {res['diversity']:<12.4f}")

print("=" * 70)

In [ ]:
# Bar chart comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

models = list(results.keys())
x = np.arange(len(models))

# Training time
times = [results[m]["training_time"] for m in models]
axes[0].bar(x, times, color="steelblue")
axes[0].set_xticks(x)
axes[0].set_xticklabels(models, rotation=45, ha="right")
axes[0].set_ylabel("Time (seconds)")
axes[0].set_title("Training Time")

# Coherence
coherences = [results[m]["coherence"] for m in models]
axes[1].bar(x, coherences, color="forestgreen")
axes[1].set_xticks(x)
axes[1].set_xticklabels(models, rotation=45, ha="right")
axes[1].set_ylabel("Coherence (UMass)")
axes[1].set_title("Topic Coherence")

# Diversity
diversities = [results[m]["diversity"] for m in models]
axes[2].bar(x, diversities, color="coral")
axes[2].set_xticks(x)
axes[2].set_xticklabels(models, rotation=45, ha="right")
axes[2].set_ylabel("Diversity")
axes[2].set_title("Topic Diversity")

plt.tight_layout()
plt.show()

## 5. Examine Topics

Look at the topics learned by each model.

In [ ]:
for name, res in results.items():
    print(f"\n{'=' * 50}")
    print(f"{name} Topics")
    print("=" * 50)
    res["model"].print_topics(top_n=5)

## 6. Model Selection Guidelines

### When to use each model:

| Model | Best For | Considerations |
|-------|----------|----------------|
| **Gibbs LDA** | Small-medium datasets, interpretability | Slower for large datasets |
| **Variational LDA** | Large datasets, speed | May converge to local optima |
| **HDP** | Unknown number of topics | Computationally expensive |
| **CTM** | Correlated topics | More parameters to tune |

## 7. Using BenchmarkRunner

For more comprehensive benchmarking, use the `BenchmarkRunner` class.

In [ ]:
# Using BenchmarkRunner for systematic comparison
runner = BenchmarkRunner(
    models=["lda_gibbs", "lda_vi"],
    num_topics_range=[3, 5],
    metrics=["coherence", "diversity"],
)

benchmark_results = runner.run(documents, num_iterations=200)

print("\nBenchmark Results:")
for result in benchmark_results:
    print(f"  {result.model_name} (K={result.num_topics}): coherence={result.coherence:.4f}")

## Next Steps

- See `03_full_tutorial.ipynb` for a complete workflow with real data
- Experiment with different hyperparameters (alpha, beta)
- Try different numbers of topics